Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)  
и задавались следующим образом:  
x - последовательность цифр  
y1 = x1,  
y(i) = x(i) + x(1).  
Если y(i) >= 10,  
то y(i) = y(i) - 10  

Задача:  
научить модель предсказывать y(i) по x(i)  
пробовать RNN, LSTM, GRU

In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def digit_encoder(x):
#    x = torch.randint(0, 10, (5, ))
    y = torch.zeros_like(x)
    
    for i in range(5):
        y[0] = x[0]
        y[i] = x[i] + x[0]
        if y[i] >= 10:
            y[i] = x[i] + x[0] -10
    return y

In [4]:
def dataset(iterance):
    X_train = torch.stack([torch.randint(0, 10, (5, )) for i in range(iterance)])
    Y_train = torch.stack([digit_encoder(i) for i in X_train])
  
    return X_train, Y_train

In [5]:
x_train, y_train = dataset(1000)

In [6]:
data_iter = DataLoader(TensorDataset(x_train, y_train),
                      batch_size=128,
                      shuffle=True)

for X, y in data_iter:
    print(X, y)
    break

RNN

In [7]:
class RNNNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(10, 10)
        self.rnn = torch.nn.RNN(10, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 10)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        out, state = self.rnn(embed)
        return self.linear(out)

In [8]:
model = RNNNetwork()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
for epoch in range(101):
    train_loss, train_passed = .0, .0

    model.train()
    for x, y in data_iter:
        y = y.view(1, -1).squeeze()
        optimizer.zero_grad()
        out = model.forward(x).view(-1, 10)
        loss = criterion(out, y)
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        train_passed += 1

    if epoch%10 == 0:
        print(f'Epoch: {epoch+1}, Train loss: {train_loss:.5f}')
    if epoch%100 == 0:    
        x = torch.randint(0, 10, (5,))
        out = model.forward(x).argmax(dim=1).view(-1)
        y = digit_encoder(x).view(-1)
        print(f' X {x}\n Y pred {out}\n Y {y}')

Epoch: 1, Train loss: 18.06309
 X tensor([7, 2, 1, 9, 6])
 Y pred tensor([7, 2, 1, 9, 6])
 Y tensor([7, 9, 8, 6, 3])
Epoch: 11, Train loss: 10.43644
Epoch: 21, Train loss: 0.46998
Epoch: 31, Train loss: 0.10671
Epoch: 41, Train loss: 0.05437
Epoch: 51, Train loss: 0.03435
Epoch: 61, Train loss: 0.02433
Epoch: 71, Train loss: 0.01834
Epoch: 81, Train loss: 0.01443
Epoch: 91, Train loss: 0.01171
Epoch: 101, Train loss: 0.00971
 X tensor([9, 0, 3, 4, 0])
 Y pred tensor([9, 9, 2, 3, 9])
 Y tensor([9, 9, 2, 3, 9])


LSTM

In [10]:
class LSTMNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(10, 10)
        self.lstm = torch.nn.LSTM(10, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 10)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        out, state = self.lstm(embed)
        return self.linear(out)

In [11]:
model = LSTMNetwork()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [12]:
for epoch in range(101):
    train_loss, train_passed = .0, .0

    model.train()
    for x, y in data_iter:
        y = y.view(1, -1).squeeze()
        optimizer.zero_grad()
        out = model.forward(x).view(-1, 10)
        loss = criterion(out, y)
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        train_passed += 1

    if epoch%10 == 0:
        print(f'Epoch: {epoch+1}, Train loss: {train_loss:.5f}')
    if epoch%100 == 0:    
        x = torch.randint(0, 10, (5,))
        out = model.forward(x).argmax(dim=1).view(-1)
        y = digit_encoder(x).view(-1)
        print(f' X {x}\n Y pred {out}\n Y {y}')

Epoch: 1, Train loss: 18.08221
 X tensor([9, 1, 8, 9, 1])
 Y pred tensor([9, 1, 8, 9, 1])
 Y tensor([9, 0, 7, 8, 0])
Epoch: 11, Train loss: 0.08618
Epoch: 21, Train loss: 0.02068
Epoch: 31, Train loss: 0.01064
Epoch: 41, Train loss: 0.00658
Epoch: 51, Train loss: 0.00452
Epoch: 61, Train loss: 0.00331
Epoch: 71, Train loss: 0.00255
Epoch: 81, Train loss: 0.00203
Epoch: 91, Train loss: 0.00166
Epoch: 101, Train loss: 0.00138
 X tensor([8, 0, 8, 9, 0])
 Y pred tensor([8, 8, 6, 7, 8])
 Y tensor([8, 8, 6, 7, 8])


GRU

In [13]:
class GRUNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(10, 10)
        self.gru = torch.nn.GRU(10, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 10)

    def forward(self, sentence, state=None):
        embed = self.embed(sentence)
        out, state = self.gru(embed)
        return self.linear(out)

In [14]:
model = GRUNetwork()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [15]:
for epoch in range(101):
    train_loss, train_passed = .0, .0

    model.train()
    for x, y in data_iter:
        y = y.view(1, -1).squeeze()
        optimizer.zero_grad()
        out = model.forward(x).view(-1, 10)
        loss = criterion(out, y)
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        train_passed += 1

    if epoch%10 == 0:
        print(f'Epoch: {epoch+1}, Train loss: {train_loss:.5f}')
    if epoch%100 == 0:    
        x = torch.randint(0, 10, (5,))
        out = model.forward(x).argmax(dim=1).view(-1)
        y = digit_encoder(x).view(-1)
        print(f' X {x}\n Y pred {out}\n Y {y}')

Epoch: 1, Train loss: 17.98467
 X tensor([1, 9, 0, 7, 4])
 Y pred tensor([1, 9, 0, 7, 4])
 Y tensor([1, 0, 1, 8, 5])
Epoch: 11, Train loss: 0.20317
Epoch: 21, Train loss: 0.03513
Epoch: 31, Train loss: 0.01776
Epoch: 41, Train loss: 0.01101
Epoch: 51, Train loss: 0.00757
Epoch: 61, Train loss: 0.00559
Epoch: 71, Train loss: 0.00431
Epoch: 81, Train loss: 0.00342
Epoch: 91, Train loss: 0.00280
Epoch: 101, Train loss: 0.00233
 X tensor([2, 2, 2, 2, 4])
 Y pred tensor([2, 4, 4, 4, 6])
 Y tensor([2, 4, 4, 4, 6])
